In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [2]:
from transformers import AutoTokenizer
import sys
import json
from scipy.special import softmax
from transformers import AutoModelForSequenceClassification
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from transformers import AutoModelForSequenceClassification, TextClassificationPipeline, AutoTokenizer, AutoConfig
import torch
import argparse
import numpy as np
import pandas as pd


/home/tajak/miniconda3/envs/emma_main/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load the test datasets

test_en = pd.read_json("../../datasets/ParlaMint-EN-CAP-test-dataset.jsonl", lines=True)
test_hr = pd.read_json("../../datasets/ParlaMint-HR-CAP-test-dataset.jsonl", lines=True)
test_ba = pd.read_json("../../datasets/ParlaMint-BA-CAP-test-dataset.jsonl", lines=True)
test_sr = pd.read_json("../../datasets/ParlaMint-RS-CAP-test-dataset.jsonl", lines=True)

print(test_en.shape, test_hr.shape, test_ba.shape,test_sr.shape)

(876, 6) (869, 6) (824, 6) (874, 6)


In [4]:
def apply_classifier(test_df, df_test_name):
	from transformers import pipeline
	print("Loading the model ...")

	cap_labels = ["Education", "Technology", "Health", "Environment", "Housing", "Labor", "Defense", "Government Operations", "Social Welfare", "Other", "Macroeconomics", "Domestic Commerce", "Civil Rights", "International Affairs", "Transportation", "Immigration", "Law and Crime", "Agriculture", "Foreign Trade", "Culture", "Public Lands", "Energy"]

	cap_model = pipeline("text-classification", model="classla/ParlaCAP-Topic-Classifier", device=0, max_length=512, truncation=True)

	texts = test_df["text"].to_list()
	preds = []

	# Classify the texts
	results = cap_model(texts)

	# Output the results
	for result in results:
		preds.append(result["label"])

	# Create a json with results

	current_results = {
		"system": "ParlaCAP-classifier",
		"predictions": [
			{
			"train": "ParlaCAP-train",
			"test": "{}".format(df_test_name),
			"predictions": preds,
			}
		]
		}

	# Save the results as a new json
	with open("submissions/submission-{}-{}.json".format("ParlaCAP", df_test_name), "w") as file:
		json.dump(current_results, file)

	print("Classification with {} on {} finished.".format("ParlaCAP", df_test_name))


In [ ]:
	# If there is CUDA out of memory issue, make the batch size smaller.
	batch_size = 600

	# split the list into batch sizes
	def split_into_batches(lst, batch_size=600):
		return [lst[i:i + batch_size] for i in range(0, len(lst), batch_size)]
	
	batches = split_into_batches(texts)

	for batch in batches:
		inputs = cap_tokenizer(batch, max_length=512, truncation=True, padding=True, return_tensors="pt").to("cuda:0")
		with torch.no_grad():
			logits = cap_model(**inputs).logits
		for idx in range(len(logits)):
			current_logit = logits[idx].tolist()
			results = transcode(current_logit, cap_labels)
			current_cap = results[0]
			preds.append(current_cap)
	

In [5]:
apply_classifier(test_en, "ParlaCAP-EN-test")

Loading the model ...
Classification with ParlaCAP on ParlaCAP-EN-test finished.


In [6]:
apply_classifier(test_hr, "ParlaCAP-HR-test")

Loading the model ...
Classification with ParlaCAP on ParlaCAP-HR-test finished.


In [9]:
apply_classifier(test_sr, "ParlaCAP-RS-test")

Loading the model ...
Classification with ParlaCAP on ParlaCAP-RS-test finished.


In [10]:
apply_classifier(test_ba, "ParlaCAP-BA-test")

Loading the model ...
Classification with ParlaCAP on ParlaCAP-BA-test finished.
